In [1]:
import cv2
import dlib
import numpy as np
import time

# Load the face detector and shape predictor
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # You will need to download this model

# Initialize variables for stability tracking
stable_count = 0
stable_duration = 4  # Number of seconds to consider stable
prev_bbox = None

# Initialize the webcam
cap = cv2.VideoCapture(0) 
# Read the image
#img = cv2.imread('marijn_smile.jpg')
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

while True:
    ret, frame = cap.read()
    
    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = face_detector(gray)
    
    for face in faces:
        landmarks = shape_predictor(gray, face)
        
        # Extract mouth landmarks
        mouth_landmarks = []
        for i in range(48, 68):  # Mouth landmarks are indexed from 48 to 67 in dlib
            x, y = landmarks.part(i).x, landmarks.part(i).y
            mouth_landmarks.append((x, y))
        
        # Calculate the bounding box around the mouth
        min_x = min(mouth_landmarks, key=lambda x: x[0])[0]
        max_x = max(mouth_landmarks, key=lambda x: x[0])[0]
        min_y = min(mouth_landmarks, key=lambda x: x[1])[1]
        max_y = max(mouth_landmarks, key=lambda x: x[1])[1]
        
        if max_x - min_x >= 224 and max_y - min_y >= 224:
            # Draw the bounding box around the mouth
            cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), (255, 255, 255), 2)

            # Check if the bounding box is relatively stable for 5 seconds
            if prev_bbox is None:
                prev_bbox = (min_x, max_x, min_y, max_y)
            else:
                if (
                    abs(prev_bbox[0] - min_x) < 10 and
                    abs(prev_bbox[1] - max_x) < 10 and
                    abs(prev_bbox[2] - min_y) < 10 and
                    abs(prev_bbox[3] - max_y) < 10
                ):
                    stable_count += 1
                    if stable_count >= stable_duration:
                        # Save a picture of the stable bounding box
                        cv2.imwrite('mouth_bounding_box.jpg', frame)
                        # Release the webcam and close all OpenCV windows
                        cap.release()
                        cv2.destroyAllWindows()
                        picture = cv2.imread('mouth_bounding_box.jpg')
                        # Crop the image to the bounding box    
                        cropped_teeth = picture[min_y:max_y, min_x:max_x]
                        # Save the cropped image
                        cv2.imwrite('cropped_mouth.jpg', cropped_teeth)
                else:
                    stable_count = 0
                prev_bbox = (min_x, max_x, min_y, max_y)
        else:
            # Display a message to "come closer to the webcam"
            cv2.putText(frame, "Come closer to the webcam", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('Mouth Bounding Box', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        



#cv2.imshow('image', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()



error: OpenCV(4.8.1) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
